# Demo 10 - Predictive Models: Regression

In this notebook we look at the basics of how to do a linear regression using various Python modules. We don't do gradient descent directly because it's easier to just use modules!

In [ ]:
# clone the course repository, change to right directory, and import libraries.
%cd /content
!git clone https://github.com/nmattei/cmps6790.git
%cd /content/cmps6790/_demos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
plt.style.use('fivethirtyeight')
# Make the fonts a little bigger in our graphs.
font = {'size'   : 20}
plt.rc('font', **font)
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['pdf.fonttype'] = 42

## Loading and Understanding the Data

For this work we will be using data from: Generalized body composition prediction equation for men using simple measurement techniques", K.W. Penrose, A.G. Nelson, A.G. Fisher, FACSM, Human Performance research Center, Brigham Young University, Provo, Utah 84602 as listed in Medicine and Science in Sports and Exercise, vol. 17, no. 2, April 1985, p. 189.

[Data availabe here.](http://staff.pubhealth.ku.dk/~tag/Teaching/share/data/Bodyfat.html)


In [ ]:
# Load the Penrose Data and have a look.
df_penrose = pd.read_csv("./data/bodyfat.csv")
df_penrose.head()

In [ ]:
# This is a bit much so let's only use a smaller part of the data for the moment.
observations = ['Age', 'Neck', 'Forearm', 'Wrist']
df_penrose[observations + ['bodyfat']].head()

In [ ]:
# How many observations do we have?
len(df_penrose)

Let's do some really basic scatter plotting...

In [ ]:
# First we iterate over the observations and see what we have.
fig, ax = plt.subplots(1, 4, figsize=(15,5))

for i,o in enumerate(observations):
    df_penrose.plot.scatter(x=o, y='bodyfat', ax=ax[i])

## Many Ways to Find a Linear Regression

Let's say we want to look at some linear regressions of single variables to see what is going on!  So let's plot some regression lines.  Note that there are at least a few different ways -- [linregress](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html), [sklearn.LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html?highlight=polynomial), [polyfit](https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html), and [statsmodels](https://www.statsmodels.org/stable/index.html).

Here's a good article about it [Data science with Python: 8 ways to do linear regression and measure their speed](https://www.freecodecamp.org/news/data-science-with-python-8-ways-to-do-linear-regression-and-measure-their-speed-b5577d75f8b/).

In [ ]:
import math
# Let's do a basic Linear Regression on a Single Variable.
# Note that linregress p-value is whether or not the slope is 0, not if the correlation is significant.
def regress_with_stats(df_penrose, observations):
  fig, ax = plt.subplots(1, 4, figsize=(15,5), sharex=False)

  for i,o in enumerate(observations):
      slope, intercept, r_value, p_value, std_err = stats.linregress(df_penrose[o],
                                                                     df_penrose['bodyfat'])
      # Pack these into a nice title
      diag_str = "p-value=%.1g\nr-value=%.3f\nstd err.=%.3f\nslope=%.3f\nintercept=%.3f" % (p_value, r_value, std_err, slope, intercept)
      df_penrose.plot.scatter(x=o, y='bodyfat', title=diag_str, ax=ax[i])
      y_pred = df_penrose[o] * slope + intercept
      # Make points and line
      pts = np.linspace(df_penrose[o].min(), df_penrose[o].max(), 500)
      line = slope * pts + intercept
      ax[i].plot(pts, line, lw=1, color='red')

      # for the curious, here is how std err of slope is computed:
      # stderr = math.sqrt(((y_pred-df_penrose['bodyfat'])**2).sum()/(len(y_pred)-2)  /  ((df_penrose['bodyfat']-df_penrose['bodyfat'].mean())**2).sum())
      # see: https://en.wikipedia.org/wiki/Ordinary_least_squares#Finite_sample_properties


regress_with_stats(df_penrose, observations)

### Which feature seems most important/predictive?

<br><br><br><br><br>

### What happens if we scale all the values?

We compute the z-score of each column using the [`scipy.stats.zscore`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.zscore.html) function

In [ ]:
from scipy.stats import zscore
df_penrose_scaled = df_penrose.apply(zscore)
# compare before and after
print('unscaled')
display(df_penrose.head(5))
print('scaled')
display(df_penrose_scaled.head(5))

In [ ]:
# now regress with scaled values.
regress_with_stats(df_penrose_scaled, observations)

**What if we want to fit a multinomial regression? I.e., use all the observations as input/independent variables**

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(df_penrose[observations],
                                                               df_penrose['bodyfat'])









## oops; linregress assumes regression on one variable only.


## Scikit-learn
Scikit-learn implements [Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) as well.

All regression/classification classes in scikit-learn assume two functions:

- `.fit(X, y)`: fit/train the model giving training data `X` (the feature matrix) and `y` (the true values)
- `.predict(X)`: given a feature matrix, return the predicted `y` values.

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
# fit the model
lr.fit(df_penrose_scaled[observations], df_penrose_scaled['bodyfat'])
# predict using the model
predictions = lr.predict(df_penrose_scaled[observations])
# compare predictions to truth
truth_and_pred = pd.DataFrame(zip(df_penrose_scaled['bodyfat'], predictions), columns=['truth', 'predicted'])
mse = ((truth_and_pred.truth-truth_and_pred.predicted)**2).sum()/len(truth_and_pred)
truth_and_pred.plot.scatter(x='truth', y='predicted', title='mse=%.2f' % mse)
truth_and_pred.head()

### Mean-squared error
$$MSE = \frac{1}{n} \sum_i (y_i - \hat{y}_i)^2$$

In [ ]:
# what are the coefficients?
# print coefficients
print('coef=', lr.coef_)
print('intercept=', lr.intercept_)
pd.DataFrame(zip(observations, lr.coef_))

### Why did the coefficients change so much?

## Let's actually make "predictions"

Up to now we've been fitting regression models on all the data and inspecting its goodness of fit. However, if we are using this as an actual predictive model, we'd want to assess how good the prediction is on **unseen** data.

In [ ]:
df_train = df_penrose_scaled.sample(frac=0.5, random_state=42)
df_test = df_penrose_scaled.drop(df_train.index)
print('%d training and %d testing samples' % (len(df_train), len(df_test)))

In [ ]:
lr = LinearRegression()
lr.fit(df_train[observations], df_train['bodyfat'])
predictions = lr.predict(df_test[observations])
truth_and_pred = pd.DataFrame(zip(df_test['bodyfat'], predictions), columns=['truth', 'predicted'])
mse = ((truth_and_pred.truth-truth_and_pred.predicted)**2).sum()/len(truth_and_pred)
truth_and_pred.plot.scatter(x='truth', y='predicted', title='mse=%.2f' % mse)


### Why is MSE higher here?

We could also use the polyfit function

In [ ]:
# Let's try to fit a linear model with PolyFit.

fig, ax = plt.subplots(1, 4, figsize=(15,5))

for i,o in enumerate(observations):
    # Fit our curve
    x1, intercept = np.polyfit(df_penrose[o],df_penrose['bodyfat'], 1)

    # Plot regular points
    df_penrose.plot.scatter(x=o, y='bodyfat', ax=ax[i])

    # Plot curve
    pts = np.linspace(df_penrose[o].min(), df_penrose[o].max(), 500)
    line = x1 * pts + intercept
    ax[i].plot(pts, line, lw=1, ls='-', color='red')

## Non-Linear Models?

So far we have only looked at the linear case -- what happens if we try to fit a polynomial to this data?

Why would we want to fit some higher order polynomial on this data? Why is it a good idea or a bad idea?

In [ ]:
# Let's try fitting a degree 2 polynomial with polyfit.

fig, ax = plt.subplots(1, 4, figsize=(15,5))

for i,o in enumerate(observations):

    # Fit the polynomial.
    x2, x1, intercept = np.polyfit(df_penrose[o],df_penrose['bodyfat'], 2)

    # Plot our points.
    df_penrose.plot.scatter(x=o, y='bodyfat', ax=ax[i])

    # Plot the Regression Line..
    pts = np.linspace(df_penrose[o].min(), df_penrose[o].max(), 500)
    line = x2 * pts**2 + x1 * pts + intercept
    ax[i].plot(pts, line, lw=1, ls='-', color='red')

In [ ]:
# Let's try fitting a degree 5 polynomial with polyfit.

fig, ax = plt.subplots(1, 4, figsize=(15,5))

for i,o in enumerate(observations):

    # Fit the polynomial.
    x5, x4, x3, x2, x1, intercept = np.polyfit(df_penrose[o],df_penrose['bodyfat'], 5)

    # Plot our points.
    df_penrose.plot.scatter(x=o, y='bodyfat', ax=ax[i])

    # Plot the Regression Line..
    pts = np.linspace(df_penrose[o].min(), df_penrose[o].max(), 500)
    line = x5 * pts**5 + x4 * pts**4 + x3 * pts**3 + x2 * pts**2 + x1 * pts + intercept
    ax[i].plot(pts, line, lw=1, ls='-', color='red')

## Regressions and Plots with StatsModels and Seaborn

Statsmodels (you'll likely need to install it) gives a much more R-like interface to linear modeling.  You can read [more about it here](https://www.statsmodels.org/stable/index.html).

In [ ]:
import statsmodels.api as sm
df_ind = df_penrose[['Neck', 'Wrist']]
df_target = df_penrose['bodyfat']

In [ ]:
X = df_ind
y = df_target

# Note the difference in argument order
# Call: endog, then exog (dependent, indepenednt)
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model
# Print out the statistics
model.summary()
#fig, ax = plt.subplots(figsize=(12,8))
#fig = sm.graphics.plot_partregress(endog="bodyfat", exog_i=['Abdomen', 'Neck'], exog_others='', data=df_penrose)

We can also use the [single regressor plot](https://tedboy.github.io/statsmodels_doc/generated/statsmodels.graphics.api.plot_partregress.html#statsmodels.graphics.api.plot_partregress).

In [ ]:
from statsmodels.graphics.regressionplots import plot_partregress
fig, ax = plt.subplots(figsize=(12,8))
plot_partregress(endog='bodyfat', exog_i='Neck', exog_others='', data=df_penrose, ax=ax)
plt.show()

If we have multiple elements in our regression then we need to use a different plot.

In [ ]:
# Multiple regression plot
from statsmodels.graphics.regressionplots import plot_partregress_grid
fig = plt.figure(figsize=(8, 6))
plot_partregress_grid(model, fig=fig)
plt.show()

Another way to work with regressions and their plots is using the [Seaborn Regression Package](https://seaborn.pydata.org/tutorial/regression.html)

In [ ]:
# Another way to do simple exploratory plots
import seaborn as sns
df_test = df_penrose.sample(frac=0.10, replace=False)
fig, ax = plt.subplots(1, 4, figsize=(15,5))

for i,o in enumerate(observations):
    sns.regplot(x=o, y='bodyfat', data=df_test, ax=ax[i])
    #g.axes.set_xlim(df_test[o].min()*.95,df_test[o].max()*1.05)



In [ ]:
# As we saw before we can also use the sns pairplot.
sns.pairplot(df_penrose[observations])

## Linear Regressions and the Bias v. Variance Tradeoff

Another nice simulator to play with is [this one](https://ndirienzo.shinyapps.io/linear_regression_sim/) which is from [Prof. Nicholas DiRienzo](https://ischool.arizona.edu/people/nicholas-dirienzo) from ASU's School of Information

## Logistic Regression

We can also use sklearn to do a quick logistic regression.  Remember that for logistic regression we are testing whether or not something is true, so we need to add a variable to our data.

Someone is obese if their body fat is >32% so we'll add a dummy for that!

In [ ]:
df_penrose['obese'] = df_penrose.apply(lambda x: 1 if x['bodyfat'] > 32 else 0, axis=1)

In [ ]:
df_penrose['obese'] = df_penrose.apply(lambda x: 1 if x['bodyfat'] > 32 else 0, axis=1)

In [ ]:
# We're going to use sklearn to build us a classifier.

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# setup our data for testing and training.

X_train, X_test, y_train, y_test = train_test_split(df_penrose[observations],
                                                    df_penrose['obese'],
                                                    test_size=0.2)


In [ ]:
X_train.head()

In [ ]:
# Fit that model!
logisticRegr = LogisticRegression(max_iter=100000, class_weight='balanced')
model = logisticRegr.fit(X_train, y_train)

In [ ]:
# Fit and plot!
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay
y_pred = model.predict(X_test)
print(f"Accuracy Score is: {accuracy_score(y_test, y_pred)}")
ConfusionMatrixDisplay.from_estimator(model, X_test, y_test,
                                        display_labels=logisticRegr.classes_,
                                        cmap=plt.cm.Blues, normalize='all')